In [ ]:
## imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
immport_prefix="/var/datasets/ImmPort/immune_RNAseq/ALLSTUDIES/ALLSTUDIES-DR49.2_Tab/Tab/"


print("WORKSPACE")
print("============================================================")
workspace = pd.read_table(f"{immport_prefix}/workspace.txt",sep="\t")
print(workspace.columns)
print(workspace.shape)
# study.head()


print("STUDY")
print("============================================================")
study = pd.read_table(f"{immport_prefix}/study.txt",sep="\t")
print(study.columns)
print(study.shape)
# study.head()

print("")
print("ARM_OR_COHORT")
print("============================================================")
arm_or_cohort = pd.read_table(f"{immport_prefix}/arm_or_cohort.txt",sep="\t")
print(arm_or_cohort.columns)
print(arm_or_cohort.shape)

print("")
print("ARM_2_SUBJECT")
print("============================================================")
arm_2_subject = pd.read_table(f"{immport_prefix}/arm_2_subject.txt",sep="\t")
print(arm_2_subject.columns)
print(arm_2_subject.shape)

# arm_2_subject.head()

print("")
print("BIOSAMPLE")
print("============================================================")
biosample = pd.read_table(f"{immport_prefix}/biosample.txt",sep="\t")
print(biosample.columns)
print(biosample.shape)

print("============================================================")
lab_test_panel = pd.read_table(f"{immport_prefix}/lab_test_panel.txt",sep="\t")
print(lab_test_panel.columns)
print(lab_test_panel.shape)

print("")
print("LAB_TEST")
print("============================================================")
lab_test = pd.read_table(f"{immport_prefix}/lab_test.txt",sep="\t")
print(lab_test.columns)
print(lab_test.shape)

print("")
print("SUBJECT")
print("============================================================")
subject = pd.read_table(f"{immport_prefix}/subject.txt", sep="\t") 
print(subject.shape)

print("ASSESSMENT_PANEL")
print("============================================================")
assessment_panel = pd.read_table(f"{immport_prefix}/assessment_panel.txt",sep="\t")
print(assessment_panel.columns)
print(assessment_panel.shape)

print("")
print("ASSESSMENT_COMPONENT")
print("============================================================")
assessment_component = pd.read_table(f"{immport_prefix}/assessment_component.txt",sep="\t")
print(assessment_component.columns)
print(assessment_component.shape)
workspace.head()

In [ ]:
#Helper methods

def lab_test_panels_by_study(study_accession,lab_tests):
    study_lab_tests = lab_tests[lab_tests['STUDY_ACCESSION']==study_accession]
    study_panels = study_lab_tests[['PANEL_NAME','LAB_TEST_NAME']]
    study_panels = study_panels.groupby(['PANEL_NAME','LAB_TEST_NAME']).size().reset_index(name="COUNTS")
    return study_panels
def assessment_panels_by_study(study_accession,assessments):
    study_assessments = assessments[assessments['STUDY_ACCESSION']==study_accession]
    study_panels = study_assessments[['PANEL_NAME','COMPONENT_NAME']]
    study_panels = study_panels.groupby(['PANEL_NAME','COMPONENT_NAME']).size().reset_index(name="COUNTS")
    return study_panels




In [ ]:
## How many Labtests
print(f'Labtests amount: {lab_test.drop_duplicates().shape}')
print(f'Labtest panels amount: {lab_test_panel.drop_duplicates().shape}')

## How many Subjects
subject = subject[subject['SPECIES']=='Homo sapiens']
print(f'Subject amount: {subject.drop_duplicates().shape}')
subject_with_gender = subject[subject['GENDER'].isin(['Male' ,'Female'])].drop_duplicates()
print(f'Subjects male/female amount: {subject_with_gender.shape}')

## How many Studies
print(f'Studies amount: {study.drop_duplicates().shape}')

## Labtests: "Biomarkers", welche 

In [ ]:
study[study['STUDY_ACCESSION']=='SDY67']

In [ ]:
study['GENDER_INCLUDED'].unique()
arm_2_subject_valid = arm_2_subject[arm_2_subject['SUBJECT_ACCESSION'].isin(subject_with_gender['SUBJECT_ACCESSION'])]
 
duplicates = arm_2_subject_valid[arm_2_subject_valid.duplicated('SUBJECT_ACCESSION', keep='first')] 
## keep says if we want to see every enry just once or all duplicates of it
print(f'Arm 2 Subject duplicates: {duplicates.shape}')
arm_2_subject_valid['AGE_UNIT'].unique() # 'Years', 'Not Specified', 'Weeks', 'Months', 'Days', 'Hours' 
### here we need to see... babys may have a month or days unit ... hours is strange for homo sapiens... 
arm_2_subject_valid = arm_2_subject_valid.dropna(subset=['MAX_SUBJECT_AGE_IN_YEARS', 'MIN_SUBJECT_AGE_IN_YEARS'])
### As we have different age events we should despite track the duplicated subjects!
print(f'Subjects valid aged male/female {arm_2_subject_valid.drop_duplicates('SUBJECT_ACCESSION').shape}')
mismatch_age = arm_2_subject_valid[arm_2_subject_valid['MAX_SUBJECT_AGE_IN_YEARS'] != arm_2_subject_valid['MIN_SUBJECT_AGE_IN_YEARS']]
print(f'Subjects with MIN / MAX age mismatch {mismatch_age.drop_duplicates('SUBJECT_ACCESSION').shape}')

arm_2_subject_valid['AGE_EVENT'].unique() # 'Age at enrollment', 'Other', 'Not Specified', 'Age at initial vaccine administration', 'Age at Study Day 0', 'Age at infection', 'Age at initial treatment'

subject.head()

In [ ]:
arm_2_subject_valid.columns

In [ ]:
subject_w_age = pd.merge(arm_2_subject_valid, subject, on='SUBJECT_ACCESSION', how='left') ### I merge into arm_2_subject_valid as we have duplicated rows here that i probably want to keep whilst in the other its unique
subject_w_age.to_csv('tables/subject_w_age.csv')
subject_w_age.columns

In [ ]:
arm_2_subject['AGE_EVENT'].unique()

In [ ]:
#### LAB TEST PANEL INSIGHTS

lab_test_panel = lab_test_panel.rename(columns={'NAME_REPORTED': 'PANEL_NAME'})
lab_test = lab_test.rename(columns={'NAME_REPORTED': 'LAB_TEST_NAME'})
lab_tests = pd.merge(lab_test_panel,lab_test,
                    left_on='LAB_TEST_PANEL_ACCESSION',right_on='LAB_TEST_PANEL_ACCESSION') 
lab_test.head()

In [ ]:
lab_tests = pd.merge(lab_test_panel,lab_test,
                    left_on='LAB_TEST_PANEL_ACCESSION',right_on='LAB_TEST_PANEL_ACCESSION')
print(lab_tests.shape)
lab_tests.to_csv('tables/lab_tests.csv')
lab_tests_backup = lab_tests.copy(deep=True)
lab_tests.head()

In [ ]:
#Helper methods
def lab_test_panels(lab_tests):
    study_panels = lab_tests[['PANEL_NAME','LAB_TEST_NAME']]
    study_panels = study_panels.groupby(['PANEL_NAME','LAB_TEST_NAME']).size().reset_index(name="COUNTS")
    return study_panels
def lab_test_panels_prefered_names(lab_tests):
    study_panels = lab_tests[['NAME_PREFERRED_x','NAME_PREFERRED_y']]
    study_panels = study_panels.groupby(['NAME_PREFERRED_x','NAME_PREFERRED_y']).size().reset_index(name="COUNTS")
    return study_panels

lab_tests_PANEL_NAME = lab_test_panels(lab_tests)
lab_tests_NAME_PREFERRED = lab_test_panels_prefered_names(lab_tests)
PANEL_NAMEs_unique = lab_tests_PANEL_NAME['PANEL_NAME'].unique()
LAB_TESTS_unique = lab_tests_PANEL_NAME['LAB_TEST_NAME'].unique()
NAME_PREFERRED_x_unique = lab_tests_NAME_PREFERRED['NAME_PREFERRED_x'].unique()
NAME_PREFERRED_y_unique = lab_tests_NAME_PREFERRED['NAME_PREFERRED_y'].unique()


print('PANEL_NAMEs '  + str(len(PANEL_NAMEs_unique)))
print('LAB_TESTS '  + str(len(LAB_TESTS_unique)))
print('LAB_TEST counts: ' + str(sum(lab_tests_PANEL_NAME['COUNTS'])))
print('PANEL_NAMEs preferred names '  + str(len(NAME_PREFERRED_x_unique)))
print('LAB_TESTS preferred names '  + str(len(NAME_PREFERRED_y_unique)))
print('LAB_TEST preferred counts: ' + str(sum(lab_tests_NAME_PREFERRED['COUNTS'])))


lab_tests_PANEL_NAME.head()

In [ ]:
print('NA NAME_PREFERRED_x: ' + str(lab_tests['NAME_PREFERRED_x'].isna().sum()))
print('NA NAME_PREFERRED_y: ' + str(lab_tests['NAME_PREFERRED_y'].isna().sum()))
print('NA PANEL_NAME: ' + str(lab_tests['PANEL_NAME'].isna().sum()))
print('NA LAB_TEST_NAME: ' + str(lab_tests['LAB_TEST_NAME'].isna().sum()))


In [ ]:
#### LIST FOR BINNING THE INDIVIDUAL STUDY TYPES!
#grouped = lab_tests_PANEL_NAME.groupby('PANEL_NAME')['LAB_TEST_NAME'].apply(list).reset_index()
#for index, row in grouped.iterrows():
"""  group = row['PANEL_NAME']
    pref_name = lab_tests[lab_tests['PANEL_NAME'] == group]['NAME_PREFERRED_x'].values
    if len(pref_name) > 0:
        pref_name = pref_name[0]
    else: 
        pref_name = ''
        
    items = row['LAB_TEST_NAME'][0]
    
    pref_item = {}
    for i, x in enumerate(row['LAB_TEST_NAME']):
        y = lab_tests[lab_tests['LAB_TEST_NAME'] == row['LAB_TEST_NAME'][i]]['NAME_PREFERRED_y'].values
        if len(y) > 0:
            pref_item[i] = y[0]
        else:
            pref_item[i] = ''
    
    print(f"\n{group} ({pref_name})")
    [print('\t' + str(x) + f' ({pref_item[i]})') for i, x in enumerate(row['LAB_TEST_NAME'])] """


In [ ]:

### biomarkers
biomarker_PANEL_NAMEs = [
    'Blood Flow Cytometry: ',
    'Protein or Enzyme Type Measurement: ',
    ]



In [ ]:
### Next step: Get all Study accessions related to the biomarker and cell type bins
### Then: Get the subject
biomarker_PANEL_NAMEs_trimmed = [x[:-2] for x in biomarker_PANEL_NAMEs]
biomarker_all_tests = lab_tests[lab_tests['PANEL_NAME'].isin(biomarker_PANEL_NAMEs_trimmed)]
biomarker_all_tests.head()


In [ ]:
biomarker_all_studies_array = biomarker_all_tests['STUDY_ACCESSION'].unique()
biomarker_all_studies = study[study['STUDY_ACCESSION'].isin(biomarker_all_studies_array)]
biomarker_all_studies.columns

In [ ]:
subject_w_age.columns ## ALLDATA subjects

In [ ]:
arm_or_cohort.columns

In [ ]:
### I take subject_w_age as I want the duplicates in this position, just need to remember to remove them for the plots
study_arm_subject = pd.merge(subject_w_age, arm_or_cohort, left_on='ARM_ACCESSION',right_on='ARM_ACCESSION')
study_arm_subject.to_csv('tables/study_arm_subject.csv')
### I expect to have the same duplicates number of duplicates as before as we kepts the different time event data: 
duplicates = study_arm_subject[study_arm_subject.duplicated('SUBJECT_ACCESSION', keep='first')] 
study_arm_subject.columns # Here I get GENDER and AGE data for subjects (SUBJECT_ACCESSION).
# Also I get related studies (STUDY_ACCESSION)


In [ ]:
#### Here I want to testwise prinst status for biomarker samples
### SUBJECT extraction
biomarker_study_arm_subject = study_arm_subject[study_arm_subject['STUDY_ACCESSION'].isin(biomarker_all_studies['STUDY_ACCESSION'])]
biomarker_study_arm_subject_no_duplicated = biomarker_study_arm_subject.drop_duplicates('SUBJECT_ACCESSION')
print(f'Amount of Subjects: {biomarker_study_arm_subject.shape}')
print(f'Amount of Subjects no duplicates: {biomarker_study_arm_subject_no_duplicated.shape}')
biomarker_study_arm_subject['AGE_EVENT'].unique() ## 'Age at initial treatment', 'Other', 'Age at enrollment', 'Age at Study Day 0'
## here aprently the age events are several, problbly for the individual studies... 
biomarker_study_arm_subject_no_duplicated

In [ ]:
### OVERALL
#sns.violinplot(x='GENDER', y='MAX_SUBJECT_AGE_IN_YEARS', hue='GENDER', data=biomarker_study_arm_subject_no_duplicated[['GENDER', 'MAX_SUBJECT_AGE_IN_YEARS']], split=True)
#plt.show()

In [ ]:
############################################################################
### Letztes meeting mit florian: only bulk Transcriptomics to start with! 
############################################################################
print("")
### Aparently this is the Gene expression data... will dig ito this a bit... 
print("EXPSAMPLE_PLUBLIC_REPOSITORY")
print("============================================================")
gene_expression = pd.read_table(f"{immport_prefix}/expsample_public_repository.txt",sep="\t")
print(gene_expression.columns)
print(gene_expression.shape)

print("EXPSAMPLE")
print("============================================================")
expsample = pd.read_table(f"{immport_prefix}/expsample.txt",sep="\t")
print(expsample.columns)
print(expsample.shape)

print("EXPERIMENT")
print("============================================================")
experiment = pd.read_table(f"{immport_prefix}/experiment.txt",sep="\t")
print(experiment.columns)
print(experiment.shape)

print("BIOSAMPLE")
print("============================================================")
biosample = pd.read_table(f"{immport_prefix}/biosample.txt",sep="\t")
print(biosample.columns)
print(biosample.shape)


print("EXPSAMPLE_2_BIOSAMPLE")
print("============================================================")
expsample_2_biosample = pd.read_table(f"{immport_prefix}/expsample_2_biosample.txt",sep="\t")
print(expsample_2_biosample.columns)
print(expsample_2_biosample.shape)



In [ ]:
## TODO measurement techniques die sinn machen in list ordnen 
####### -> damit DB reduzieren und dann NAME und DESCRIPTION nehmen um weiter zu arbeiten
####### -> ergebniss evtl nochmal kurz mit flo absprechen wenn Zeit...
experiment['MEASUREMENT_TECHNIQUE'].unique()
valid_measurement_techiques = [
    'SNP microarray',
    'Transcription profiling by array',
    'Other',
    'Sequencing',
    'Flow Cytometry',
    'Q-PCR',
    'PCR',
    'CyTOF',
    'Liquid Chromatography',
    'Nanostring',
    'microRNA profiling assay',
    'Array',
    'Transcription profiling assay',
    'Real time polymerase chain reaction assay',
    'LC_MS',
    'in situ Hybridization', 
    'DNA microarray'
                               ]


## aparently DNA microarray can also be used for RNA 
## Liquid Chromatography -> es gibt wohl ne LC technik die RNAseq kann


## Flow Cytometry -> eher protein aber lass ich mal drin... 
## Immunoblot = Westernblot -> Proteine (Northern Blot wäre RNA)
## PCR -> reverse transcription: 
## Real-time PCR, on the other hand, is a more targeted approach to gene expression analysis
## Cytometric Bead Array Assay , SOMAscan assay -> proteins
## Olink assay, Multiplex Bead Array Assay -> protein biomarkers

## EMSA -> Protein - RNA interaction

## Whole Genome Sequencing , DNA microarray -> DNA 


In [ ]:
subject['SPECIES'].unique()

In [ ]:
experiment_transcriptomes = experiment[experiment['MEASUREMENT_TECHNIQUE'].isin(valid_measurement_techiques)]
### ALSO i need to filter for homo sapiens ... 
workspace_ids_homo_sapiens = subject[subject['SPECIES'] == 'Homo sapiens']['WORKSPACE_ID'].unique()
experiment_transcriptomes_human = experiment_transcriptomes[experiment_transcriptomes['WORKSPACE_ID'].isin(workspace_ids_homo_sapiens)]
experiment_transcriptomes_human.shape

In [ ]:
### Hier wieder ne List aus der ich händisch raus sortien will welche methoden nicht passen
################################
### Ab hier mess ich BULK 
## kein CyTOF, Other -> ich mach mal nur microarray und sequencing und erweitere des stück für stück ... 

experiment_transcriptomes_sorted = experiment_transcriptomes_human.sort_values(by='MEASUREMENT_TECHNIQUE')
for name, group in experiment_transcriptomes_sorted.groupby('MEASUREMENT_TECHNIQUE'):
    print(f"{name}")
    for _, row in group.iterrows():
        print(f"\t {row['NAME']} ({row['DESCRIPTION']})")
    print() 
    
    
sorted_out_measurements = [
    'CyTOF',
    'Other',
    '',
    '',
]

sorted_out_names = [
    'VT2020',
    'Bacterial re-isolation (nan)',
    '',
    '',
]

In [ ]:
## Raus nehmen :     'SNP microarray', 'DNA microarray', 
include_measurements = [
    'Transcription profiling by array',
    'Sequencing',
    'Array',
]
experiment_transcriptomes_sorted_ma_seq = experiment_transcriptomes_sorted[experiment_transcriptomes_sorted['MEASUREMENT_TECHNIQUE'].isin(include_measurements)]
experiment_transcriptomes_sorted_ma_seq['MEASUREMENT_TECHNIQUE'].unique()

In [ ]:
experiment_transcriptomes_sorted_ma_seq_experiments = experiment_transcriptomes_sorted_ma_seq['EXPERIMENT_ACCESSION'].unique()
studies_transcriptomes_sorted_ma_seq_experiments = experiment_transcriptomes_sorted_ma_seq['STUDY_ACCESSION'].unique()

print('BULK RNA expression data: ')
print(f'Studies: {len(studies_transcriptomes_sorted_ma_seq_experiments)}')
print(f'Experiments: {len(experiment_transcriptomes_sorted_ma_seq_experiments)}')
print(f'Subjects: ')


In [ ]:
expsample_gene_expr = pd.merge(expsample, gene_expression, on='EXPSAMPLE_ACCESSION', how='inner')
expsample_gene_expr.head()

In [ ]:
## Filter only for valid experiments! 
expsample_gene_expr_bulk_transriptomes = expsample_gene_expr[expsample_gene_expr['EXPERIMENT_ACCESSION'].isin(experiment_transcriptomes_sorted_ma_seq_experiments)]
print(expsample_gene_expr_bulk_transriptomes.shape)
print(expsample_gene_expr_bulk_transriptomes.columns)

In [ ]:
subject_w_age.columns

In [ ]:
### Kills the kernel
""" subject_w_age['WORKSPACE_ID'] = subject_w_age['WORKSPACE_ID'].astype(int)
expsample_gene_expr_bulk_transriptomes_w_age = pd.merge(expsample_gene_expr_bulk_transriptomes, subject_w_age, on='WORKSPACE_ID', how='inner')
print(expsample_gene_expr_bulk_transriptomes_w_age.columns)
expsample_gene_expr_bulk_transriptomes_w_age.head() """

In [ ]:
biosample_subject_accession = biosample[['BIOSAMPLE_ACCESSION', 'SUBJECT_ACCESSION']]
mapping_df_biosample_expsample = pd.merge(expsample_2_biosample, biosample_subject_accession, on='BIOSAMPLE_ACCESSION', how='inner')
print(mapping_df_biosample_expsample.columns)


expsample_gene_expr_bulk_transriptomes_mapping = pd.merge(expsample_gene_expr_bulk_transriptomes, mapping_df_biosample_expsample, on='EXPSAMPLE_ACCESSION', how='inner')
print(expsample_gene_expr_bulk_transriptomes.columns)

subject_w_age['WORKSPACE_ID'] = subject_w_age['WORKSPACE_ID'].astype(int)
expsample_gene_expr_bulk_transriptomes_mapping['WORKSPACE_ID'] = expsample_gene_expr_bulk_transriptomes_mapping['WORKSPACE_ID'].astype(int)
expsample_gene_expr_bulk_transriptomes_w_subjects = pd.merge(expsample_gene_expr_bulk_transriptomes_mapping, subject_w_age, on='SUBJECT_ACCESSION', how='inner') 
print(expsample_gene_expr_bulk_transriptomes_w_subjects.columns)
expsample_gene_expr_bulk_transriptomes_w_subjects.head()


In [ ]:
counts = expsample_gene_expr_bulk_transriptomes_w_subjects.groupby('SUBJECT_ACCESSION')['REPOSITORY_ACCESSION'].count()
counts_df = counts.reset_index(name='count')

counts_df.describe()
max_outlier = counts_df[counts_df['count']>= 500]
max_outlier
#expsample_gene_expr_bulk_transriptomes_w_subjects[expsample_gene_expr_bulk_transriptomes_w_subjects['SUBJECT_ACCESSION'] == 'SUB134248']['EXPSAMPLE_ACCESSION'].unique()



In [ ]:
print(f'Shape expsamples {expsample_gene_expr_bulk_transriptomes_w_subjects.shape}')
subjects_unique_df = expsample_gene_expr_bulk_transriptomes_w_subjects.drop_duplicates('SUBJECT_ACCESSION', keep='first')
print(subjects_unique_df.shape)
subjects_unique_list = expsample_gene_expr_bulk_transriptomes_w_subjects['SUBJECT_ACCESSION'].unique()
print(f'Subjects {len(subjects_unique_list)}')


In [ ]:
sns.violinplot(x='GENDER', y='MAX_SUBJECT_AGE_IN_YEARS', hue='GENDER', data=subjects_unique_df[['GENDER', 'MAX_SUBJECT_AGE_IN_YEARS']], split=True)
plt.show()

In [ ]:
### This seems to kill my kernel all over again. 
#expsample_gene_expr_bulk_transriptomes_w_subject = pd.merge(expsample_gene_expr_bulk_transriptomes, subject, on='WORKSPACE_ID', how='left')
#expsample_gene_expr_bulk_transriptomes_w_subject.head()

In [ ]:
expsample_gene_expr_bulk_transriptomes_w_subjects

In [ ]:
expsample_gene_expr_bulk_transriptomes_w_subjects['REPOSITORY_ACCESSION'].unique()
#### GEO: GSM specifies a sample usually
### Test GEO: GSM1885263 ; Test SRA: 


In [ ]:
expsample_gene_expr_bulk_transriptomes_w_subjects['REPOSITORY_NAME'].unique()

In [ ]:
### As we only get GSM IDs i need a script to automatically download stuff from GEO (and maybe the other platforms too  ?)
from Bio import Entrez

# Set your email address for NCBI Entrez
Entrez.email = "elias.schreiner@uni-tuebingen.de"

# Function to search for datasets on GEO
def search_geo(query):
    handle = Entrez.esearch(db="gds", term=query, retmax=10)
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

# Function to download datasets by accession number
def download_geo(accession):
    handle = Entrez.efetch(db="gds", id=accession, rettype="xml")
    record = Entrez.read(handle)
    handle.close()
    # Extract download links from the record and download the datasets
    for link in record[0]["EXPERIMENT_PACKAGE"]["SUPPLEMENTARY_FILE"]:
        url = link["@url"]
        filename = os.path.basename(url)
        print(f"Downloading {filename}...")
        os.system(f"wget {url}")

accession_list = search_geo('GSM1885263')
print(accession_list)
#download_geo(accession_list[0])